In [18]:
import torch
from torch import nn
from torchvision import models, datasets, transforms
import time
from tqdm.auto import tqdm
import os

In [19]:
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

input_size = 224
batch_size = 4
normalize = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

device = torch.device("cuda:0")
num_classes = 10

criterion = nn.CrossEntropyLoss()

def train_model(model, dataloaders, criterion, optimizer, num_epochs=3):
    since = time.time()

    acc_history = {'train': [], 'val': []}
    loss_history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model, acc_history

def test_model(model, testloader, criterion):
    n_batches = len(testloader)
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(testloader, total=n_batches):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / len(testloader.dataset)
    acc = running_corrects.double() / len(testloader.dataset)
    print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(loss, acc))
    return acc, loss

In [20]:
# Load model

resnet18_model = models.resnet18(pretrained=True)
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)

In [21]:
# Load dataset

# download dataset and unpack:
dataset_path = "imagenette2"
if not os.path.exists(dataset_path):
    !wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
    !tar xvzf imagenette2.tgz

trainset_imagenette = datasets.ImageFolder(root='./%s/train/' % dataset_path, transform=normalize)
trainloader_imagenette = torch.utils.data.DataLoader(trainset_imagenette, batch_size=batch_size, shuffle=True, num_workers=2)

testset_imagenette = datasets.ImageFolder(root='./%s/val/' % dataset_path, transform=normalize)
testloader_imagenette = torch.utils.data.DataLoader(testset_imagenette, batch_size=batch_size, shuffle=False, num_workers=2)

loaders_imagenette = {'train': trainloader_imagenette, 'val': testloader_imagenette}

In [22]:
train_optimizer = torch.optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9)
resnet18_model = resnet18_model.to(device)

In [23]:
# train model
train_model(resnet18_model, loaders_imagenette, criterion, train_optimizer, num_epochs=5)

Epoch 0/4
----------



train Loss: 0.5149 Acc: 0.8437



val Loss: 0.1692 Acc: 0.9442

Epoch 1/4
----------



train Loss: 0.3283 Acc: 0.8989



val Loss: 0.2365 Acc: 0.9304

Epoch 2/4
----------


...


train Loss: 0.1810 Acc: 0.9460



val Loss: 0.1475 Acc: 0.9564

Epoch 4/4
----------



train Loss: 0.1218 Acc: 0.9653



val Loss: 0.2135 Acc: 0.9409

Training complete in 15m 31s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [24]:
# test model
test_model(resnet18_model, testloader_imagenette, criterion)


Val Loss: 0.2135 Val Acc: 0.9409


(tensor(0.9409, device='cuda:0', dtype=torch.float64), 0.21352363121768278)

In [25]:
# save fc layer trained on imagenette
imagenette_fc = resnet18_model.fc

In [26]:
trainset_cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=normalize)
trainloader_cifar10 = torch.utils.data.DataLoader(trainset_cifar10, batch_size=batch_size, shuffle=True, num_workers=2)

testset_cifar10 = datasets.CIFAR10(root='./data', train=False, download=True, transform=normalize)
testloader_cifar10 = torch.utils.data.DataLoader(testset_cifar10, batch_size=batch_size, shuffle=False, num_workers=2)

loaders_cifar10 = {'train': trainloader_cifar10, 'val': testloader_cifar10}

Files already downloaded and verified
Files already downloaded and verified


In [27]:
num_ftrs = resnet18_model.fc.in_features
set_requires_grad(resnet18_model, False)
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)
resnet18_model = resnet18_model.to(device)

In [28]:
pretrain_optimizer = torch.optim.SGD(resnet18_model.fc.parameters(), lr=0.001, momentum=0.9)
train_model(resnet18_model, loaders_cifar10, criterion, pretrain_optimizer, num_epochs=2)
# Train
set_requires_grad(resnet18_model, True)
train_optimizer = torch.optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9)
train_model(resnet18_model, loaders_cifar10, criterion, train_optimizer, num_epochs=1)

Epoch 0/1
----------



train Loss: 1.4310 Acc: 0.5175



val Loss: 0.9704 Acc: 0.6782

Epoch 1/1
----------



train Loss: 1.3259 Acc: 0.5625



val Loss: 1.2858 Acc: 0.6110

Training complete in 10m 45s
Epoch 0/0
----------



train Loss: 0.7387 Acc: 0.7653



val Loss: 0.3483 Acc: 0.8816

Training complete in 13m 28s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [29]:
# test model on cifar10
test_model(resnet18_model, testloader_cifar10, criterion)


Val Loss: 0.3483 Val Acc: 0.8816


(tensor(0.8816, device='cuda:0', dtype=torch.float64), 0.3482725682939581)

In [30]:
# restore fc layer
resnet18_model.fc = imagenette_fc
resnet18_model = resnet18_model.to(device)

In [31]:
# test model on imagenette2
test_model(resnet18_model, testloader_imagenette, criterion)


Val Loss: 1.2535 Val Acc: 0.5992


(tensor(0.5992, device='cuda:0', dtype=torch.float64), 1.2534793946332043)

In [32]:
# train only last layer on imagenette2
set_requires_grad(resnet18_model, False)
set_requires_grad(resnet18_model.fc, True)
train_model(resnet18_model, loaders_imagenette, criterion, train_optimizer, num_epochs=5)

Epoch 0/4
----------



train Loss: 1.0169 Acc: 0.6799



val Loss: 0.7749 Acc: 0.7636

Epoch 1/4
----------



train Loss: 1.0237 Acc: 0.6816



val Loss: 0.7107 Acc: 0.7740

Epoch 2/4
----------


...


train Loss: 1.0353 Acc: 0.6735



val Loss: 0.8348 Acc: 0.7432

Epoch 4/4
----------



train Loss: 1.0310 Acc: 0.6764



val Loss: 0.7210 Acc: 0.7710

Training complete in 11m 21s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [33]:
# test model on imagenette2
test_model(resnet18_model, testloader_imagenette, criterion)


Val Loss: 0.7210 Val Acc: 0.7710


(tensor(0.7710, device='cuda:0', dtype=torch.float64), 0.7210337683662271)

Итоги:
1. Трейн на imagenette2: Val Loss: 0.2135 Val Acc: 0.9409
2. После дообучения нового последнего слоя на CIFAR10: Val Loss: 0.3483 Val Acc: 0.8816
3. После возврата слоя тест на imagenette2: Val Loss: 1.2535 Val Acc: 0.5992
4. После дообучения возвращенного слоя на imagenette2: Val Loss: 0.7210 Val Acc: 0.7710
Исходное качество не достигнуто (accuracy было 0.9409, стало 0.7710).
Вывод: после замены слоя, обучения на другом датасете и возврата слоя сеть перестает обучаться нормально на исходном датасете. Тем самым подтверждается эффект катастрофического забывания.